# 🍓 Task 3: Matching Strawberry to Peduncle

This notebook trains a **Siamese Network** to determine if a specific Strawberry belongs to a specific Peduncle (stem).
This is crucial for robotics to know which stem to cut for a target fruit.

**Method:**
1.  **Download Dataset** (Github Releases).
2.  **Generate Pairs**: Create positive pairs (True Parent) and negative pairs (Random Other Peduncle).
3.  **Siamese Model**: Shared Encoder + Contrastive/Binary Head.
4.  **Training**: Minimize Binary Cross Entropy.

## 1. Setup & Download

Standard setup block.

In [ ]:
import os
import requests
import zipfile
import shutil
import json
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

try:
    import timm
except ImportError:
    !pip install -q timm
    import timm

# --- Configuration ---
GITHUB_TYPE = "releases"
VERSION_TAG = "v1.0"
BASE_URL = f"https://github.com/SergKurchev/strawberry_synthetic_dataset/releases/download/{VERSION_TAG}"
FILES_TO_DOWNLOAD = [
    "strawberry_dataset.zip.001",
    "strawberry_dataset.zip.002",
    "strawberry_dataset.zip.003"
]
OUTPUT_ZIP = "strawberry_dataset.zip"
DATASET_ROOT = Path("strawberry_dataset")

def setup_dataset():
    search_paths = [
        Path("strawberry_dataset"),
        Path("dataset/strawberry_dataset"),
        Path("/kaggle/input/last-straw-dataset/strawberry_dataset"),
        Path("/kaggle/input/strawberry_synthetic_dataset/strawberry_dataset")
    ]
    for p in search_paths:
        if p.exists() and (p / "annotations.json").exists():
            print(f"✅ Dataset found at: {p}")
            return p

    print("⬇️ Dataset not found. Downloading...")
    os.makedirs("temp_download", exist_ok=True)
    for filename in FILES_TO_DOWNLOAD:
        file_path = Path("temp_download") / filename
        if not file_path.exists():
            r = requests.get(f"{BASE_URL}/{filename}", stream=True)
            with open(file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192): f.write(chunk)

    print("📦 Combining...")
    with open(OUTPUT_ZIP, 'wb') as outfile:
        for filename in FILES_TO_DOWNLOAD:
            with open(Path("temp_download") / filename, 'rb') as infile: shutil.copyfileobj(infile, outfile)

    print("📂 Extracting...")
    with zipfile.ZipFile(OUTPUT_ZIP, 'r') as zip_ref: zip_ref.extractall(".")
    
    shutil.rmtree("temp_download")
    os.remove(OUTPUT_ZIP)
    return DATASET_ROOT

DATASET_PATH = setup_dataset()
if not DATASET_PATH: raise RuntimeError("Setup failed")

## 2. Generate Pairs

We need pairs of images (Strawberry, Peduncle) and a label (1=Match, 0=No Match).

In [ ]:
with open(DATASET_PATH / "annotations.json", 'r') as f: coco = json.load(f)

# Organize data
img_to_anns = defaultdict(list)
for ann in coco['annotations']:
    img_to_anns[ann['image_id']].append(ann)

pairs = []

CROPS_BASE = Path("matching_crops")
CROPS_BASE.mkdir(exist_ok=True)
(CROPS_BASE / "straw").mkdir(exist_ok=True)
(CROPS_BASE / "ped").mkdir(exist_ok=True)

print("🖇️ Generating Pairs...")

for img_info in tqdm(coco['images']):
    img_id = img_info['id']
    if img_id not in img_to_anns: continue

    img_path = DATASET_PATH / "images" / img_info['file_name']
    if not img_path.exists(): continue
    img_cv = cv2.imread(str(img_path))
    if img_cv is None: continue

    anns = img_to_anns[img_id]
    
    # Extract all objects first
    straws = []
    peds = [] # Dict: id -> path

    for ann in anns:
        x, y, w, h = [int(v) for v in ann['bbox']]
        # Pad
        pad=5
        x, y = max(0, x-pad), max(0, y-pad)
        w, h = min(img_cv.shape[1]-x, w+2*pad), min(img_cv.shape[0]-y, h+2*pad)
        crop = img_cv[y:y+h, x:x+w]
        if crop.size==0: continue
        
        save_name = f"{img_id}_{ann['id']}.jpg"
        
        if ann['category_id'] in [0, 1, 2]: # Strawberry
            p = CROPS_BASE / "straw" / save_name
            cv2.imwrite(str(p), crop)
            straws.append({"ann": ann, "path": p})
        elif ann['category_id'] == 3: # Peduncle
            p = CROPS_BASE / "ped" / save_name
            cv2.imwrite(str(p), crop)
            peds.append({"ann": ann, "path": p})

    # Make positive pairs
    ped_map = {p['ann']['instance_id']: p for p in peds}
    
    for s in straws:
        parent_id = s['ann'].get('parent_id', 0)
        if parent_id in ped_map:
            # Positive
            pairs.append((str(s['path']), str(ped_map[parent_id]['path']), 1.0))
            
            # Negative (Hard mining: same image, different peduncle)
            other_peds = [pid for pid in ped_map.keys() if pid != parent_id]
            if other_peds:
                neg_pid = random.choice(other_peds)
                pairs.append((str(s['path']), str(ped_map[neg_pid]['path']), 0.0))

print(f"✅ Generated {len(pairs)} pairs.")

## 3. Siamese Network Training

Dual-input network.

In [ ]:
# Dataset
class SiameseDataset(Dataset):
    def __init__(self, pairs, transform=None):
        self.pairs = pairs
        self.transform = transform
        
    def __len__(self): return len(self.pairs)
    
    def __getitem__(self, idx):
        p1, p2, label = self.pairs[idx]
        img1 = Image.open(p1).convert("RGB")
        img2 = Image.open(p2).convert("RGB")
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2, torch.tensor(label, dtype=torch.float32)

# Model
class SiameseNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Shared Encoder
        self.encoder = timm.create_model('resnet18', pretrained=True, num_classes=0)
        
        # Classification Head
        self.head = nn.Sequential(
            nn.Linear(512 * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x1, x2):
        f1 = self.encoder(x1)
        f2 = self.encoder(x2)
        combined = torch.cat([f1, f2], dim=1)
        return self.head(combined).squeeze()

# Setup
train_pairs, val_pairs = train_test_split(pairs, test_size=0.2, random_state=42)

tf = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_loader = DataLoader(SiameseDataset(train_pairs, tf), batch_size=32, shuffle=True)
val_loader = DataLoader(SiameseDataset(val_pairs, tf), batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

print("🚀 Training Siamese Network...")
for epoch in range(5):
    model.train()
    loss_acc = 0
    for x1, x2, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x1, x2)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        loss_acc += loss.item()
    print(f"  Loss: {loss_acc/len(train_loader):.4f}")

print("✅ Training Complete")

## 4. Evaluate

Check accuracy.

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for x1, x2, y in val_loader:
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        out = model(x1, x2)
        preds = (out > 0.5).float()
        correct += (preds == y).sum().item()
        total += y.size(0)

print(f"🎯 Validation Accuracy: {correct/total*100:.2f}%")